# Task A

In this task, we had to create a "transform" function which transforms unperformed beats time into a performed version.

In other words, given the `midi_score_annotations.txt` and `NAME_annotations.txt`, we need to create a function that transfers first column of the first file into the first column of the second file.

For the velocity, we use `*.mid` files and extract velocity from events. "Beat velocity" is defined as an average of the velocities of the events close to this beat. Event is close to the beat iff the difference between its onset in score and beat time is smaller than `0.5`. Then we again create a "transform" function.

In [ ]:
%load_ext autoreload
%autoreload 2

from src.data import get_dataset_metadata, get_midi_performance_pairs, get_events_table_from_score, DATASET_PATH
from src.plots import plot_average_transfer_function
from src.estimators import random_estimator, random_velocity_estimator
from src.onset_distribution import compute_average_distribution

In [ ]:
df, json_data = get_dataset_metadata("Bach")

In [ ]:
df.head(2)

In [ ]:
# for the first time might take up to 7 min because of velocity
midi_beats_list, velocity_beats_list, performance_beats_list, perf_velocity_beats_list = get_midi_performance_pairs(df, json_data)

In [ ]:
performance_beats_estimated_list = []
velocity_beats_estimated_list = []
for midi_beats, velocity_beats in zip(midi_beats_list, velocity_beats_list):
    performance_beats_estimated = random_estimator(midi_beats)
    velocity_beats_estimated = random_velocity_estimator(velocity_beats)
    performance_beats_estimated_list.append(performance_beats_estimated)
    velocity_beats_estimated_list.append(velocity_beats_estimated)

In [ ]:
plot_average_transfer_function(midi_beats_list, performance_beats_list, performance_beats_estimated_list, performance_type="time")
plot_average_transfer_function(midi_beats_list, perf_velocity_beats_list, velocity_beats_estimated_list, performance_type="velocity")

# Task B

Use `src.data.get_events_table_from_score`

In [ ]:
# We analyse Bach's Fugues (or other?)
corpus = DATASET_PATH / "Bach" / "Fugue"
# TODO : fetch the time signature from the XML (<time>), and group by time signature to perform one experiment for each
four_four = [
    "bwv_846",
    "bwv_848",
]
score_paths = [corpus / piece / "midi_score.mid" for piece in four_four]
compute_average_distribution(score_paths, sig=(4,4), subdivision=4)